In [3]:
from __future__ import division

from pyomo.environ import *

model=ConcreteModel()

    
Y='y'
A='a'
a=[]
y=[]

for i in range(1,31):
    if i <10:
        y.append(Y+'0'+str(i))
    else:
        y.append(Y+str(i))

for i in range(1,25):
    if i <10:
        a.append(A+'0'+str(i))
    else:
        a.append(A+str(i))
        

c=0
I=[]
for i in range(1,10):
    I.append(c+i)
    

#Sets    
model.t=Set(initialize=y,ordered=True,doc='time periods')

model.a=Set(initialize=a,ordered=True,doc='age of trees')

model.m=Set(initialize=['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec'],ordered=True,doc='month')

model.m_s=Set(initialize=['tobacco','maize','timber-1','timber-2','other'],ordered=True)

model.mc=Set(initialize=['jul','aug','sep','oct'],ordered=True,doc='cutting months')

model.i=Set(initialize=I,doc='annualy',ordered=True)

model.c=Set(initialize=['maize','tobacco'],ordered=True)

model.c_s=Set(initialize=['input-cost','yield','price'],ordered=True)

#Tables

Table_ld={ 
 ('maize', 'jan'): 46.7,
 ('maize', 'feb'): 10.0,
 ('maize', 'mar'): 13.0,
 ('maize', 'apr'): 16.7,
 ('maize', 'may'): 18.3,
 ('maize', 'jun'): 0,
 ('maize', 'jul'): 0,
 ('maize', 'aug'): 0,
 ('maize', 'sep'): 0,
 ('maize', 'oct'): 0,
 ('maize', 'nov'): 13.3,
 ('maize', 'dec'): 33.3,
 ('other', 'jan'): 10.0, 
 ('other', 'feb'): 8.0,
 ('other', 'mar'): 12.0,  
 ('other', 'apr'): 10.0,
 ('other', 'may'): 4.0,
 ('other', 'jun'): 2.0, 
 ('other', 'jul'): 0,
 ('other', 'aug'): 0,
 ('other', 'sep'): 0,
 ('other', 'oct'): 10.0, 
 ('other', 'nov'): 28.0, 
 ('other', 'dec'): 24.0,
 ('timber-1', 'jan'): 10.0,
 ('timber-1', 'feb'): 0,
 ('timber-1', 'mar'): 3.0,
 ('timber-1', 'apr'): 0,
 ('timber-1', 'may'): 10.0,
 ('timber-1', 'jun'): 0,
 ('timber-1', 'jul'): 0,
 ('timber-1', 'aug'): 1.0,
 ('timber-1', 'sep'): 7.0,
 ('timber-1', 'oct'): 8.0,
 ('timber-1', 'nov'): 9.0,  
 ('timber-1', 'dec'): 0,
 ('timber-2', 'jan'): 7.0,
 ('timber-2', 'feb'): 0,
 ('timber-2', 'mar'): 0,
 ('timber-2', 'apr'): 0,
 ('timber-2', 'may'): 10.0,
 ('timber-2', 'jun'): 0,
 ('timber-2', 'jul'): 0,
 ('timber-2', 'aug'): 1.0,
 ('timber-2', 'sep'): 0,      
 ('timber-2', 'oct'): 0,
 ('timber-2', 'nov'): 0,
 ('timber-2', 'dec'): 0,
 ('tobacco', 'jan'): 44.7,
 ('tobacco', 'feb'): 84.2,
 ('tobacco', 'mar'): 87.3,
 ('tobacco', 'apr'): 90.7,
 ('tobacco', 'may'): 20.5,
 ('tobacco', 'jun'): 0,
 ('tobacco', 'jul'): 0,
 ('tobacco', 'aug'): 0,
 ('tobacco', 'sep'): 37.3,
 ('tobacco', 'oct'): 44.7,
 ('tobacco', 'nov'): 19.7,
 ('tobacco', 'dec'): 46.4,
 }

model.ld=Param(model.m_s,model.m,initialize=Table_ld,mutable=True,doc='labor requirements')

#crop data
Table_tmd = {('input-cost', 'maize'): 20,
             ('input-cost', 'tobacco'): 2000,
             ('price', 'maize'): 0.8,
             ('price', 'tobacco'): 8.4,
             ('yield', 'maize'): 750,
             ('yield', 'tobacco'): 675}

model.tmd=Param(model.c_s,model.c,initialize=Table_tmd,mutable=True,doc='crop data')


#Scalars
model.nfam=Param(initialize=400,doc='number of families in village')
model.sfam=Param(initialize=3,doc='size of family (adult equiv)')
model.resw=Param(initialize=0.5,doc='reservetion wage (tsh per hour)')
model.fdmaize=Param(initialize=650,doc='domestic maize demand (kg per family)')
model.tob=Param(initialize=234,doc='steady state tobacco (ha)')
model.sr=Param(initialize=5,doc='starting radius of annulus (km)')
model.width=Param(initialize=1,doc='width of annulus (km)')
model.ws=Param(initialize=5,doc='walking speed (km per hour)')
model.whd=Param(initialize=10,doc='work hours per day')
model.wdm=Param(initialize=25,doc='working days per month')
model.tr=Param(initialize=1.31,doc='transport cost (tsh per m3 per km)')
model.tc=Param(initialize=349,doc='timber planting cost (tsh per ha)')
model.wrc=Param(initialize=46.75,doc='wood for curing (m3 per ha)')
model.dwr=Param(initialize=10,doc='domestic wood requirement (m3 per family)')

model.labwc=Param(initialize=1,doc='labor for cutting forest (man-days per m3)')
model.labvc=Param(initialize=0.5,doc='labor for cutting timber (man-days per m3)')

model.yw=Param(initialize=40,doc='yeild for existing forest (m3 per ha)')
model.rho=Param(initialize=0.1,doc='discount rate')
model.matr=Param(initialize=0.25,doc='improvement in maize yield after tobacco')

#Scalars initialize

def dmaize_init(model):
    return model.fdmaize * model.nfam

def wr_init(model):
    return model.tob * model.wrc + model.dwr * model.nfam

model.dmaize=Param(initialize=dmaize_init,doc='domestic maize demand in village (kg)')
model.wr=Param(initialize=wr_init,doc='wood requirements for domestic use and tobacco curing (m3)')


#ord_set declaration se morfi parameter

def ord_t_init(model):
    d={}
    for k,i in enumerate(model.t):
        d[i]=k+1
    return d

model.ord_t=Param(model.t,initialize=ord_t_init)

def ord_a_init(model):
    d={}
    for k,i in enumerate(model.a):
        d[i]=k+1
    return d

model.ord_a=Param(model.a,initialize=ord_a_init)

def ord_i_init(model):
    d={}
    for k,i in enumerate(model.i):
        d[i]=k+1
    return d

model.ord_i=Param(model.i,initialize=ord_i_init)


#Parameter initialize

a_values=[]        
for i in range(1,25):
    a_values.append(0)
    
yv_values=zip(a,a_values)

yv_init=dict(yv_values)

def replace_value_in_dict(key_to_find, definition):
    for key in yv_init.keys():
        if key == key_to_find:
            yv_init[key] = definition            
for i in ['a08','a16','a24']:
    replace_value_in_dict(i,120)
    
def delt_init(model,t): return (1+model.rho)**(-model.ord_t[t])

def delta_init(model,a): return (1+model.rho)**(-model.ord_a[a])

def vr_init(model,t):
    return model.tr*model.sr*sum(model.yv[a]*model.delta[a] for a in model.a if model.ord_t[t]+model.ord_a[a]>len(model.t))
        
def dist_init(model,i): return model.sr - model.width/2 + model.width*model.ord_i[i]

def fa_init(model,i): return 100*3.1416*model.dist[i]*model.width

def labor_init(model,i):
    return model.nfam*(model.sfam*model.wdm-model.ld['other',i])  #kalw ti seira 'other' tou Table_ld

def labw_init(model,i): return model.yw*model.labwc*model.whd/(model.whd-2*model.dist[i]/model.ws)

def cc_init(model,i): return model.tmd['input-cost',i]

def yc_init(model,i): return model.tmd['yield',i]

def pc_init(model,i): return model.tmd['price',i]

def cr_init(model,i): return model.yc[i]*model.pc[i]

    
#Parameters

model.yv=Param(model.a,initialize=yv_init,mutable=True,doc='yield of planted timber (m3 per ha)')
print model.yv.keys()

model.delt=Param(model.t,initialize=delt_init,mutable=True,doc='discaount factor')

model.delta=Param(model.a,initialize=delta_init,mutable=True,doc='discaount factor for tree age')

model.vr=Param(model.t,initialize=vr_init,mutable=True,doc='residual velue of timber (tsh)')

model.dist=Param(model.i,initialize=dist_init,mutable=True,doc='average distance of annulus (km)')

model.fa=Param(model.i,initialize=fa_init,mutable=True,doc='area of annulus (ha)')

model.labor=Param(model.m,initialize=labor_init,mutable=True,doc='village labor supply (man-days)')

model.labw=Param(model.i,initialize=labw_init,mutable=True,doc='labor for cutting forest (man-days)')

model.cc=Param(model.c,initialize=cc_init,mutable=True,doc='input cost (tsh per ha)')

model.yc=Param(model.c,initialize=yc_init,mutable=True,doc='crop yields (kg per ha)')

model.pc=Param(model.c,initialize=pc_init,mutable=True,doc='crop prices (tsh per kg)')

model.cr=Param(model.c,initialize=cr_init,mutable=True,doc='crop revenues (tsh per ha)')


#Variables

model.w=Var(model.t,model.i,within=NonNegativeReals,doc='cutting of existing forest (ha)')

model.v=Var(model.t,within=NonNegativeReals,doc='management of planted timber (ha)')

model.x=Var(model.t,model.c,within=NonNegativeReals,doc='cropping activity (ha)')

model.mat=Var(model.t,within=NonNegativeReals,doc='maize after tobacco (ha)')

model.lc=Var(model.t,model.m,within=NonNegativeReals,doc='labor for wood cutting (man-days)')

model.rev=Var(model.t,doc='revenue (1000 tsh)')

model.cost=Var(model.t,doc='annual cost (1000 tsh)')


#Constraints

def lag_lead(model,model_onoma_set,onoma_set,value):  
    l=[i for i in model_onoma_set]
    indx=l.index(onoma_set)
    m=indx+value
    if m>-1 and m<len(l):
        return l[m]
    else:
        return None
    
    
def wb_rule(model,t):     
    return model.yw*sum(model.w[t,i] for i in model.i)+sum(model.yv[a]*dict(model.v).get(lag_lead(model,model.t,t,-model.ord_a[a]),0) for a in model.a)>=model.wrc*model.x[t,'tobacco']+model.dwr*model.nfam

for t in model.t:
    print wb_rule(model,t)

model.wb=Constraint(model.t,rule=wb_rule,doc='wood balance')

def wa_rule(model,i): return sum(model.w[t,i] for t in model.t)<=model.fa[i]
 
model.wa=Constraint(model.i,rule=wa_rule,doc='wood availability')

def lb_rule(model,t,m):
    return model.ld['timber-1',m]*model.v[t] + model.ld['timber-2',m]*dict(model.v).get(lag_lead(model,model.t,t,-1),0)+sum(model.ld[c,m]*model.x[t,c] for c in model.c)+( m in model.mc and 1 or 0)*model.lc[t,m] <=model.labor[m]

  
model.lb=Constraint(model.t,model.m,rule=lb_rule,doc='labor balance') 

def lw_rule(model,t): 
    return sum(model.lc[t,mc] for mc in model.mc )==sum(model.labw[i]*model.w[t,i] for i in model.i)+sum(model.labvc*model.yv[a]*dict(model.v).get(lag_lead(model,model.t,t,-model.ord_a[a]),0) for a in model.a)
 
model.lw=Constraint(model.t,rule=lw_rule,doc='labor constraint for wood cutting')

def mm_rule(model,t):
    return model.yc['maize']*(model.x[t,'maize']+model.matr*model.mat[t])>=model.dmaize

model.mm=Constraint(model.t,rule=mm_rule,doc='minimum maize demand')

def ttb_rule(model,t):
    return sum(model.yv[a]*dict(model.v).get(lag_lead(model,model.t,t,(len(model.t)-model.ord_a[a])),0) for a in model.a)<=model.wr
  
model.ttb=Constraint(model.t,rule=ttb_rule,doc='post terminal timber bounds')

def matd1_rule(model,t):
    l=[i for i in model.t]
    indx=l.index(t) 
    return model.mat[t]<=dict(model.x).get((lag_lead(model,model.t,t,-1),'tobacco'),0)

model.matd1=Constraint(model.t,rule=matd1_rule,doc='maize after tobacco : type 1')   

def matd2_rule(model,t):
    return model.mat[t]<=model.x[t,'maize']

model.matd2=Constraint(model.t,rule=matd2_rule,doc='maize after tobacco : type 2')

def rd_rule(model,t):
    return model.rev[t]==(sum(model.cr[c]*model.x[t,c] for c in model.c)+model.matr*model.cr['maize']*model.mat[t])/1000

model.rd=Constraint(model.t,rule=rd_rule,doc='revenue definition')

def cd_rule(model,t):
    return model.cost[t]==(sum(model.cc[c]*model.x[t,c] for c in model.c)+model.tc*model.v[t]+model.resw*model.whd*sum(model.lc[t,mc]for mc in model.mc)+sum(model.tr*model.yw*model.dist[i]*model.w[t,i] for i in model.i))/1000

model.cd=Constraint(model.t,rule=cd_rule,doc='cost definition')

def matup_init(model):
    return model.mat['y01']<=model.tob

model.matup=Constraint(rule=matup_init,doc='mat[y01] upper bound')

def od_rule(model):
    return sum(model.delt[t]*(model.rev[t]-model.cost[t]+model.vr[t]*model.v[t]/1000) for t in model.t)
   
model.od=Objective(rule=od_rule,doc='objective definition',sense=maximize)

['a20', 'a21', 'a08', 'a09', 'a24', 'a02', 'a03', 'a01', 'a06', 'a07', 'a04', 'a05', 'a23', 'a15', 'a14', 'a17', 'a16', 'a11', 'a10', 'a13', 'a12', 'a19', 'a18', 'a22']
4000 + 46.75*x[y01,tobacco]  <=  40 * ( w[y01,1] + w[y01,2] + w[y01,3] + w[y01,4] + w[y01,5] + w[y01,6] + w[y01,7] + w[y01,8] + w[y01,9] )
4000 + 46.75*x[y02,tobacco]  <=  40*( w[y02,1] + w[y02,2] + w[y02,3] + w[y02,4] + w[y02,5] + w[y02,6] + w[y02,7] + w[y02,8] + w[y02,9] ) + yv[a01] * v[y01]
4000 + 46.75*x[y03,tobacco]  <=  40*( w[y03,1] + w[y03,2] + w[y03,3] + w[y03,4] + w[y03,5] + w[y03,6] + w[y03,7] + w[y03,8] + w[y03,9] ) + yv[a01] * v[y02] + yv[a02] * v[y01]
4000 + 46.75*x[y04,tobacco]  <=  40*( w[y04,1] + w[y04,2] + w[y04,3] + w[y04,4] + w[y04,5] + w[y04,6] + w[y04,7] + w[y04,8] + w[y04,9] ) + yv[a01] * v[y03] + yv[a02] * v[y02] + yv[a03] * v[y01]
4000 + 46.75*x[y05,tobacco]  <=  40*( w[y05,1] + w[y05,2] + w[y05,3] + w[y05,4] + w[y05,5] + w[y05,6] + w[y05,7] + w[y05,8] + w[y05,9] ) + yv[a01] * v[y04] + yv[a02] *